In [2]:
import pandas as pd
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from scipy import stats
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
import matplotlib.dates as mdates
from scipy import stats
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import median_absolute_error
from sklearn.decomposition import PCA, IncrementalPCA

In [ ]:
df=pd.read_csv("Fractionator.csv")


In [ ]:
df.columns


In [ ]:
good_data=df[(np.abs(stats.zscore(df["AI_2020_OVERHEAD_C5'S_MOL"])) < 3)]

In [ ]:
from datetime import date
df['Time']=pd.to_datetime(df['Time'])

In [ ]:
manipulated_variables=["AI_2020_OVERHEAD_C5'S_MOL", "AI_2021_MIDDLE_C7'S_MOL",
       "AI_2022_BOTTOM_C3'S_MOL"]
controlled_variables=[
       'FIC-2001_SP_TOP_REFLUX_SETPOINT_MBBL/D',
       'FIC-2001_OP_TOP_REFLUX_OUTPUT_%',
       'FIC-2002_SP_MIDDLE_PRODUCT_DRAW_SETPOINT_MBBL/D',
       'FIC_2002_OP_MIDDLE_PRODUCT_DRAW_OUTPUT_%',
       'FIC_2004_SP_MIDDLE_REFLUX _SETPOINT_MBBL/D',
       'FIC_2004_OP_MIDDLE_REFLUX_OUTPUT_%', 
       'TIC_2003_SP_FEED_TEMPERATURE_SETPOINT_DEGF']
reponse_variables=['FIC_2100_PV_FEED_FURNACE_FUEL_SCFH',
       'FIC-2101_PV_TOP_PRODUCT_MBBL/D', 'FIC_2102_PV_BOTTOM_PRODUCT_MBBL/D',
       'FI-2005_PV_FEED_FLOW_MBBL/D',
       'FIC_2001_PV_TOP_REFLUX_SETPOINT_MBBL/D',
       'FIC_2002_PVMIDDLE_PRODUCT_DRAW_MBBL/D',
       'FIC_2004_PV_TOP_REFLUX_MBBL/D',
       'QI_2106_PV_MIDDLE_REFLUX_DUTY_BTU/H',]

In [ ]:
list(enumerate(manipulated_variables))
for i in enumerate(controlled_variables):
    print(df[controlled_variables].columns[i[0]])

In [ ]:
plt.figure(figsize = (15, 30))
for i in enumerate(controlled_variables):
    plt.subplot(8, 1,i[0]+1)
    plt.plot(df["Time"],df[i[1]])
plt.show()

In [ ]:
plt.figure(figsize = (15, 30))
for i in enumerate(manipulated_variables):
    plt.subplot(3, 1,i[0]+1)
    plt.plot(df["Time"],df[i[1]])
    plt.title(df[manipulated_variables].columns[i[0]])
    plt.xlabel('Time')
    plt.ylabel(df[manipulated_variables].columns[i[0]])
   # plt.gcf().autofmt_xdate()
    hours = mdates.HourLocator(interval=6)
    hours_fmt = mdates.DateFormatter('%H-%D')
    plt.gca().xaxis.set_major_locator(hours)
    plt.gca().xaxis.set_major_formatter(hours_fmt)
    plt.xticks(rotation = 45)


plt.show()

In [ ]:
plt.figure(figsize = (15, 30))
for i in enumerate(manipulated_variables):
    plt.subplot(3, 1,i[0]+1)
    plt.plot(good_data["Time"],good_data[i[1]])
    plt.title(good_data[manipulated_variables].columns[i[0]])
    plt.xlabel('Time')
    plt.ylabel(good_data[manipulated_variables].columns[i[0]])
   # plt.gcf().autofmt_xdate()
    hours = mdates.HourLocator(interval=6)
    hours_fmt = mdates.DateFormatter('%H-%D')
    plt.gca().xaxis.set_major_locator(hours)
    plt.gca().xaxis.set_major_formatter(hours_fmt)
    plt.xticks(rotation = 45)


plt.show()

In [ ]:
FIC_reflux_top=['FIC-2001_SP_TOP_REFLUX_SETPOINT_MBBL/D','FIC-2001_OP_TOP_REFLUX_OUTPUT_%','FIC_2001_PV_TOP_REFLUX_SETPOINT_MBBL/D']
FIC_reflux_middle=[ 'FIC-2002_SP_MIDDLE_PRODUCT_DRAW_SETPOINT_MBBL/D',
       'FIC_2002_OP_MIDDLE_PRODUCT_DRAW_OUTPUT_%',
       'FIC_2002_PVMIDDLE_PRODUCT_DRAW_MBBL/D']
FIC_reflux_bottom=['FIC_2004_SP_MIDDLE_REFLUX _SETPOINT_MBBL/D',
       'FIC_2004_OP_MIDDLE_REFLUX_OUTPUT_%', 'FIC_2004_PV_TOP_REFLUX_MBBL/D']
furnace=['FIC_2100_PV_FEED_FURNACE_FUEL_SCFH','TIC_2003_SP_FEED_TEMPERATURE_SETPOINT_DEGF']

In [ ]:
plt.figure(figsize = (15, 5))
for i in enumerate(furnace):
    plt.plot(df["Time"],df[i[1]])
    hours = mdates.HourLocator(interval=6)
    hours_fmt = mdates.DateFormatter('%H-%D')
    plt.gca().xaxis.set_major_locator(hours)
    plt.gca().xaxis.set_major_formatter(hours_fmt)
    plt.xticks(rotation = 45)
    plt.yticks([0,25,50,195,198,200,202,205])
plt.show()

In [ ]:
plt.figure(figsize = (15, 5))
for i in enumerate(FIC_reflux_middle):
    plt.plot(good_data["Time"],good_data[i[1]])
    hours = mdates.HourLocator(interval=6)
    hours_fmt = mdates.DateFormatter('%H-%D')
    plt.gca().xaxis.set_major_locator(hours)
    plt.gca().xaxis.set_major_formatter(hours_fmt)
    plt.xticks(rotation = 45)
    #plt.yticks([0,25,50,195,198,200,202,205])
plt.show()

In [ ]:
target=df[manipulated_variables]
data=df[controlled_variables+reponse_variables]

In [ ]:
target=target.to_numpy()
data=data.to_numpy()

In [ ]:
X=good_data.iloc[:,4:]

In [ ]:
y=good_data.iloc[:,1:4]